In [1]:
import numpy as np
import xarray as xr
from xarray import DataArray
import matplotlib.pyplot as plt
from virtual_ecosystem.example_data.generation_scripts.common import (
    time_index,
    x_cell_ids,
    y_cell_ids,
)
path = "/Users/lin/Code/ve_example"

In [8]:
ds = xr.open_dataset(path + "/source/ERA5_land.nc")
ds = ds.rename({
    "longitude": "x",
    "latitude": "y", 
    "time": "time_index",
    "tp": "precipitation"
})
ds

<xarray.Dataset> Size: 308kB
Dimensions:        (x: 20, y: 20, time_index: 24)
Coordinates:
  * x              (x) float32 80B 116.1 116.2 116.3 116.4 ... 117.8 117.9 118.0
  * y              (y) float32 80B 6.0 5.9 5.8 5.7 5.6 ... 4.5 4.4 4.3 4.2 4.1
  * time_index     (time_index) datetime64[ns] 192B 2013-01-01 ... 2014-12-01
Data variables:
    d2m            (time_index, y, x) float64 77kB ...
    t2m            (time_index, y, x) float64 77kB ...
    sp             (time_index, y, x) float64 77kB ...
    precipitation  (time_index, y, x) float64 77kB ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-17 10:55:00 GMT by grib_to_netcdf-2.24.0: /opt/ecmw...

In [5]:
ds = xr.open_dataset(path + "/source/ERA5_land.nc")
ds = ds.rename({
    "longitude": "x",
    "latitude": "y", 
    "time": "time_index",
    "tp": "precipitation"
})
ds = ds.assign_coords({"x": np.arange(0, 1800, 90), "y": np.arange(0, 1800, 90), "time_index": time_index})
ds = ds.isel(x=x_cell_ids, y=y_cell_ids)
ds

<xarray.Dataset> Size: 63kB
Dimensions:        (time_index: 24, y: 9, x: 9)
Coordinates:
  * x              (x) int64 72B 0 90 180 270 360 450 540 630 720
  * y              (y) int64 72B 0 90 180 270 360 450 540 630 720
  * time_index     (time_index) int64 192B 0 1 2 3 4 5 6 ... 18 19 20 21 22 23
Data variables:
    d2m            (time_index, y, x) float64 16kB ...
    t2m            (time_index, y, x) float64 16kB ...
    sp             (time_index, y, x) float64 16kB ...
    precipitation  (time_index, y, x) float64 16kB ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-17 10:55:00 GMT by grib_to_netcdf-2.24.0: /opt/ecmw...

In [7]:
TotalPrecp50 = xr.open_dataset("/Users/lin/Code/Project/TotalPrecp_1975-2024.nc")
TotalPrecp50 = TotalPrecp50.sel(
    latitude=slice(6, 4.1),  
    longitude=slice(116.1, 118)
)
TotalPrecp50 = TotalPrecp50.rename({
    "longitude": "x",
    "latitude": "y", 
    "valid_time": "time_index",
    "tp": "precipitation"
})
TotalPrecp50

<xarray.Dataset> Size: 975kB
Dimensions:        (time_index: 600, y: 20, x: 20)
Coordinates:
    number         int64 8B ...
  * time_index     (time_index) datetime64[ns] 5kB 1975-01-01 ... 2024-12-01
  * y              (y) float64 160B 6.0 5.9 5.8 5.7 5.6 ... 4.5 4.4 4.3 4.2 4.1
  * x              (x) float64 160B 116.1 116.2 116.3 ... 117.8 117.9 118.0
    expver         (time_index) <U4 10kB ...
Data variables:
    precipitation  (time_index, y, x) float32 960kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-19T13:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
TotalPrecp50 = xr.open_dataset("/Users/lin/Code/Project/TotalPrecp_1975-2024.nc")
TotalPrecp50 = TotalPrecp50.sel(
    latitude=slice(6, 4.1),  
    longitude=slice(116.1, 118)
)
TotalPrecp50 = TotalPrecp50.rename({
    "longitude": "x",
    "latitude": "y", 
    "valid_time": "time_index",
    "tp": "precipitation"
})
TotalPrecp50 = TotalPrecp50.assign_coords({"x": np.arange(0, 1800, 90), "y": np.arange(0, 1800, 90)})
TotalPrecp50 = TotalPrecp50.isel(x=x_cell_ids, y=y_cell_ids)
TotalPrecp50['precipitation'].values = TotalPrecp50['precipitation'].values * 1000 * 30

TotalPrecp50


<xarray.Dataset> Size: 209kB
Dimensions:        (time_index: 600, y: 9, x: 9)
Coordinates:
    number         int64 8B ...
  * time_index     (time_index) datetime64[ns] 5kB 1975-01-01 ... 2024-12-01
    expver         (time_index) <U4 10kB ...
  * x              (x) int64 72B 0 90 180 270 360 450 540 630 720
  * y              (y) int64 72B 0 90 180 270 360 450 540 630 720
Data variables:
    precipitation  (time_index, y, x) float32 194kB 234.5 253.0 ... 165.3 181.9
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-19T13:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
yearly_total_precip = TotalPrecp50['precipitation'].groupby('time_index.year').sum(dim=['time_index', 'y', 'x'])
df = yearly_total_precip.to_dataframe(name='total_precip').reset_index().rename(columns={'year': 'Year'})
df_sorted = df.sort_values(by='total_precip', ascending=False).reset_index(drop=True)
df_sorted

,Year,number,total_precip
0,2008,0,263325.562500
1,2021,0,252390.500000
2,2017,0,249735.875000
3,2010,0,237778.687500
4,1977,0,234962.125000
5,1984,0,233866.093750
6,1999,0,228733.968750
7,2024,0,228226.093750
8,2011,0,227610.484375
9,2007,0,227390.187500


In [14]:
middle_year_sets = {
    "Middle_1322": [2013, 2022],
    "Middle_7580": [1975, 1980],
    "Middle_9620": [1996, 2020],
    "Middle_8179": [1981, 1979],
    "Middle_7600": [1976, 2000],
    "Middle_1214": [2012, 2014],
    "Middle_9578": [1995, 1978],
    "Middle_8518": [1985, 2018],
    "Middle_8688": [1986, 1988],
    "Middle_0193": [2001, 1993],
    "Middle_0916": [2009, 2016],
    "Middle_8919": [1989, 2019],
    "Middle_1598": [2015, 1998],
    "Middle_8306": [1983, 2006],
    "Middle_9187": [1991, 1987],
}

for name, years in middle_year_sets.items():
    selected_precip = TotalPrecp50['precipitation'].sel(
        time_index=TotalPrecp50['time_index'].dt.year.isin(years)
    )
    
    ds_copy = ds.copy()
    ds_copy['precipitation'].values = selected_precip.values
    ds_copy.load()

    for var in ds_copy.data_vars:
        ds_copy[var].encoding.clear()
    
    ds_copy.to_netcdf(f"{path}/data/{name}.nc")

In [10]:
ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2002) |  
                                              (TotalPrecp50['time_index'].dt.year == 1990))
Low_0290 = ds
Low_0290.load()
for var in Low_0290.data_vars:
   Low_0290[var].encoding.clear()
Low_0290.to_netcdf(path + '/data/Low_0290.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2023) |  
                                              (TotalPrecp50['time_index'].dt.year == 2002))
Low_2304 = ds
Low_2304.load()
for var in Low_2304.data_vars:
   Low_2304[var].encoding.clear()
Low_2304.to_netcdf(path + '/data/Low_2304.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2005) |  
                                              (TotalPrecp50['time_index'].dt.year == 1992))
Low_0592 = ds
Low_0592.load()
for var in Low_0592.data_vars:
   Low_0592[var].encoding.clear()
Low_0592.to_netcdf(path + '/data/Low_0592.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2003) |  
                                              (TotalPrecp50['time_index'].dt.year == 1997))
Low_0397 = ds
Low_0397.load()
for var in Low_0397.data_vars:
   Low_0397[var].encoding.clear()
Low_0397.to_netcdf(path + '/data/Low_0397.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 1994) |  
                                              (TotalPrecp50['time_index'].dt.year == 1982))
Low_9482 = ds
Low_9482.load()
for var in Low_9482.data_vars:
   Low_9482[var].encoding.clear()
Low_9482.to_netcdf(path + '/data/Low_9482.nc')

In [11]:
ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2008) |  
                                              (TotalPrecp50['time_index'].dt.year == 2021))
High_0821 = ds
High_0821.load()
for var in High_0821.data_vars:
    High_0821[var].encoding.clear()
# High_0821
High_0821.to_netcdf(path + '/data/High_0821.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2017) |  
                                              (TotalPrecp50['time_index'].dt.year == 2010))
High_1710 = ds
High_1710.load()
for var in High_1710.data_vars:
    High_1710[var].encoding.clear()
High_1710.to_netcdf(path + '/data/High_1710.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 1977) |  
                                              (TotalPrecp50['time_index'].dt.year == 1984))
High_7784 = ds
High_7784.load()
for var in High_7784.data_vars:
    High_7784[var].encoding.clear()
High_7784.to_netcdf(path + '/data/High_7784.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 1999) |  
                                              (TotalPrecp50['time_index'].dt.year == 2024))
High_9924 = ds
High_9924.load()
for var in High_9924.data_vars:
    High_9924[var].encoding.clear()
High_9924.to_netcdf(path + '/data/High_9924.nc')

ds['precipitation'].values = TotalPrecp50['precipitation'].sel(time_index=(TotalPrecp50['time_index'].dt.year == 2011) |  
                                              (TotalPrecp50['time_index'].dt.year == 2007))
High_1107 = ds
High_1107.load()
for var in High_1107.data_vars:
    High_1107[var].encoding.clear()
High_1107.to_netcdf(path + '/data/High_1107.nc')

In [15]:
from virtual_ecosystem.models.hydrology.constants import HydroConsts
hc = HydroConsts()
for field in hc.__dataclass_fields__:
    print(f"{field}: {getattr(hc, field)}")

soil_moisture_residual: 0.1
soil_moisture_saturation: 0.6
saturated_hydraulic_conductivity: 0.001
hydraulic_gradient: 0.01
van_genuchten_nonlinearily_parameter: 2.0
stream_flow_capacity: 5000.0
intercept_parameters: (0.935, 0.498, 0.00575)
veg_density_param: 0.046
groundwater_capacity: 500
bypass_flow_coefficient: 1.0
air_entry_water_potential: -3.815
campbell_pore_size_distribution: -7.22
extinction_coefficient_global_radiation: 0.7
max_percolation_rate_uzlz: 1
groundwater_loss: 1
reservoir_const_upper_groundwater: 20
reservoir_const_lower_groundwater: 20
initial_aerodynamic_resistance_surface: 12.5
initial_aerodynamic_resistance_canopy: 12.5
drag_coefficient_evaporation: 0.2
intercept_residence_time: 86400.0
initial_stomatal_conductance: 1000.0
pore_connectivity_parameter: 0.5
air_entry_potential_inverse: 0.1
m_to_kpa: 9.804


In [13]:
from virtual_ecosystem.core.constants import CoreConsts
hc = CoreConsts()
for field in hc.__dataclass_fields__:
    print(f"{field}: {getattr(hc, field)}")

placeholder: 123.4
zero_Celsius: 273.15
standard_pressure: 101.325
standard_mole: 44.642
molar_heat_capacity_air: 29.19
gravity: 6.6743e-11
stefan_boltzmann_constant: 5.6703744191844314e-08
von_karmans_constant: 0.4
max_depth_of_microbial_activity: 0.25
soil_moisture_capacity: 0.5
meters_to_mm: 1000.0
molecular_weight_air: 28.96
gas_constant_water_vapour: 461.51
seconds_to_day: 86400.0
characteristic_dimension_leaf: 0.01
specific_gas_constant_dry_air: 287.05
molecular_weight_ratio_water_to_dry_air: 0.622
conductance_to_resistance_conversion_factor: 40.9


In [ ]:
# def distribute_monthly_rainfall(
#     total_monthly_rainfall: NDArray[np.float32],
#     num_days: int,
#     seed: int | None = None,
# ) -> NDArray[np.float32]:
#     """Distributes total monthly rainfall over the specified number of days.

#     At the moment, this function allocates each millimeter of monthly rainfall to a
#     randomly selected day. In the future, this allocation could be based on observed
#     rainfall patterns.

#     Args:
#         total_monthly_rainfall: Total monthly rainfall, [mm]
#         num_days: Number of days to distribute the rainfall over
#         seed: Seed for random number generator, optional

#     Returns:
#         An array containing the daily rainfall amounts, [mm]
#     """
#     rng = np.random.default_rng(seed)

#     daily_rainfall_data = []
#     for rainfall in total_monthly_rainfall:
#         daily_rainfall = np.zeros(num_days)

#         for _ in range(int(rainfall)):
#             day = rng.integers(0, num_days, seed)  # Randomly select a day
#             daily_rainfall[day] += 1.0  # Add 1.0 mm of rainfall to the selected day

#         daily_rainfall *= rainfall / np.sum(daily_rainfall)
#         daily_rainfall_data.append(daily_rainfall)

#     return np.nan_to_num(np.array(daily_rainfall_data), nan=0.0)


# def distribute_monthly_rainfall(
#     monthly_total: NDArray[np.float32],
#     num_days: int,
#     k: float = 0.1,  # not sure, maybe need to calculate,
#     theta: float = 1,  # because I haven't find the papar provide suggeted value
#     wet_day_percent: float = 0.7,
# ) -> np.ndarray:
#     rng = np.random.default_rng()
#     n_wet = int(num_days * wet_day_percent)
#     rainfall = rng.gamma(k, theta, size=n_wet)

#     daily = np.zeros(num_days)
#     wet_indices = rng.choice(num_days, size=n_wet, replace=False)
#     daily[wet_indices] = rainfall

#     daily *= monthly_total / np.sum(daily)
    #  return np.nan_to_num(np.array(daily), nan=0.0)
